#### Vasculogenesis quantification analysis code

Data load & Preprocessing

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu, gaussian
from skimage import morphology, measure
from scipy.ndimage import binary_fill_holes, convolve
import cv2
import numpy as np
import os

def data_load(path):
    
    image = cv2.imread(path)
    if image is None:
        print("Error: Image cannot be loaded. Please check the path.")

    
        
    # Finding the index of the dominant channel
    # For BGR, 0 = Blue, 1 = Green, 2 = Red
    # We want to map this to your scale where 2 = Red, 1 = Green, 0 = Blue
    #rgb_code = np.argmax(channel_sum)  # This will be 0, 1, or 2
    #channel_sum = np.sum(image, axis=(0, 1))  # Summing over height and width for each channel
    #image_array = image[:,:,rgb_code]
    image_array = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image_array



def preprocessing_image(image, thresh_value, white_min, black_min, gaussian_blur):
    """이미지 전처리 최적화"""

    # ✅ NumPy 연산 최적화 (불필요한 중복 변수 제거)
    binary_image = image > thresh_value

    # ✅ `morphology.remove_small_objects()` 최적화 (연산 속도 개선)
    cleaned_image = morphology.remove_small_objects(binary_image, min_size=white_min)
    inverted_image = ~cleaned_image  # NOT 연산 최적화
    filtered_black_holes = morphology.remove_small_objects(inverted_image, min_size=black_min)
    final_binary_image = ~filtered_black_holes

    # ✅ Gaussian Blur 최적화
    blurred_image = gaussian(final_binary_image, sigma=gaussian_blur)
    binary_blurred_image = blurred_image > threshold_otsu(blurred_image)

    # ✅ Skeletonization 최적화
    skeleton_original = morphology.skeletonize(binary_blurred_image)
    skeleton_dilated = dilation(skeleton_original, disk(2))

    return skeleton_dilated, binary_blurred_image, skeleton_original





c:\Users\seub1\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\seub1\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\seub1\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
import networkx as nx
import cv2
import numpy as np
from skimage.measure import label, regionprops
import scipy.ndimage as ndi

def check_directions(neighborhood, pos):
    """
    Check for the presence of '1's in the direct north, south, east, and west of a central point
    in a 3x3 neighborhood.

    Parameters:
        neighborhood (numpy.ndarray): 3x3 array where the central point is checked.
        pos (tuple): The position (row, col) of the central point within the neighborhood.

    Returns:
        dict: Dictionary containing boolean values for each direction (north, south, east, west).
    """
    y, x = pos
    directions = {
        'north': neighborhood[y-1, x] if y > 0 else False,
        'south': neighborhood[y+1, x] if y < 2 else False,
        'east':  neighborhood[y, x+1] if x < 2 else False,
        'west':  neighborhood[y, x-1] if x > 0 else False
    }
    return directions


def node_seperate(skel):
    """Find branch points in a skeletonized image."""
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])
    neighbors_count = convolve(skel.astype(np.uint8), kernel, mode='constant', cval=0)
    return (neighbors_count > 2) & skel



import numpy as np
from scipy.ndimage import convolve

def find_endpoints(skel):
    """
    Find endpoints in a skeletonized image, then erase endpoints below the cutoff range on the y-axis.
    
    Parameters:
        skel (np.ndarray): Skeletonized image.
        cutoff (float): Proportion of the y-axis to keep. Default is 0.9.
        
    Returns:
        int: Number of endpoints after applying the cutoff.
        np.ndarray: Binary array indicating endpoint positions after applying the cutoff.
    """
    cutoff=0.9
    # Define the endpoint kernel
    endpoint_kernel = np.array([[1, 1, 1],
                                [1, 10, 1],
                                [1, 1, 1]])
    
    # Perform convolution to find endpoints
    neighbors = convolve(skel.astype(np.uint8), endpoint_kernel, mode='constant', cval=0)
    
    # Detect endpoints
    endpoints = (neighbors == 11)
    
    # Calculate the cutoff index for the y-axis
    y_cutoff = int(skel.shape[0] * cutoff)
    
    # Create a mask to zero out endpoints below the cutoff range
    mask = np.zeros_like(skel, dtype=bool)
    mask[:y_cutoff, :] = True
    
    # Apply the mask to the endpoints
    endpoints_cut = np.where(mask, endpoints, 0)
    
    # Count the number of endpoints after applying the cutoff
    num_endpoints = np.sum(endpoints_cut)
    
    return num_endpoints, endpoints_cut



def find_branchpoints(skeleton):
    """
    Find branch points in a skeletonized image array based on specified rules.
    
    Parameters:
        skeleton (numpy.ndarray): A binary image where the skeleton is True (1) and the background is False (0).
    
    Returns:
        numpy.ndarray: An array of the same shape as `skeleton` with branch points marked as True.
    """
    
    rows, cols = skeleton.shape
    branch_points = np.zeros_like(skeleton, dtype=bool)

    # Iterate over each pixel excluding the borders
    for y in range(1, rows-1):
        for x in range(1, cols-1):
            # Extract 3x3 neighborhood
            neighborhood = skeleton[y-1:y+2, x-1:x+2].copy() * 1
            
            
            # Check if the center is a skeleton point and has exactly three 1s in the neighborhood
            if neighborhood[1, 1] == 1 and np.sum(neighborhood) == 4:  # 4 because it includes the center point
                # Set the center to 0 to check non-adjacency of other ones
                
                neighborhood[1, 1] = 0

                # Find the positions of ones
                ones_positions = np.argwhere(neighborhood)
                
                # Check each 1's 3x3 neighborhood to ensure no other 1s are adjacent
                valid = True
                for pos in ones_positions:
                    if sum(check_directions(neighborhood, pos).values()) != 0:
                        valid = False
                        break

                if valid:
                    branch_points[y, x] = True

    return branch_points

# Function to calculate average thickness
def calculate_thickness(image, skeleton):
    distance = ndi.distance_transform_edt(image)
    skeleton_distance = skeleton * distance
    thickness = skeleton_distance[skeleton_distance > 0]
    return np.mean(thickness), np.std(thickness)

def mesh_numbering(image) : 
    image = image.astype(np.uint8) * 255
    """input data type np.uint8 0 ~ 255 narray"""
    contours, hierarchy = cv2.findContours(image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    fourth_column = hierarchy[0][:, 3]

    return len(fourth_column[fourth_column != -1])


### 지훈이형 데이터 원본 이미지 / 전처리 이미지 비교용

In [3]:
### 25.02.11 수정버전 num_pixels -> area

def node_analysis(skeleton_blurred_image, binary_blurred_image) :

    # mesh 마다 label 넘버링 및 정량화 단계
    labeled_meshes_skeleton = label(skeleton_blurred_image, connectivity=2)
    labeled_meshes_vessel = label(binary_blurred_image, connectivity=2)
    
    # 전체 영역의 properties 계산
    properties = regionprops(labeled_meshes_skeleton)
    properties_vessel = regionprops(labeled_meshes_vessel)
    
    # area를 기준으로 properties 정렬
    # properties와 properties_vessel을 함께 정렬하기 위해 인덱스 리스트를 만들어 정렬
    indices = list(range(len(properties_vessel)))
    indices.sort(key=lambda i: properties_vessel[i].area, reverse=True)
    
    properties = [properties[i] for i in indices]
    properties_vessel = [properties_vessel[i] for i in indices]
    
    # node 개수(branch point) 계산
    node_points = find_branchpoints(skeleton_blurred_image)
    
    # Calculate thickness
    thickness_mean, thickness_std = calculate_thickness(binary_blurred_image, skeleton_blurred_image)
    total_endpoint_number, total_endpoint_position = find_endpoints(skeleton_blurred_image)
    
    node_list = {}
    node_list['0'] = { 'total_area': [], 'total_density':[], 'total_node_number': [], 'total_node_position':[], 'total_node_length': [], 'total_endpoint_number': [], 'total_endpoint_position': [], 'connectivity': [], 'total_mesh_number' : [], 'total_thickness (mean ± std)':[], 'total_vessel_image': [], 'total_vessel_skel_image':[] }
    node_list['0']['total_area'] = np.count_nonzero(binary_blurred_image)
    node_list['0']['total_density'] = np.sum(binary_blurred_image) / binary_blurred_image.size
    node_list['0']['total_node_number'] = np.sum(node_points)
    node_list['0']['total_node_position'] = node_points
    node_list['0']['total_node_length'] = np.sum(skeleton_blurred_image)
    node_list['0']['total_endpoint_number'] = total_endpoint_number
    node_list['0']['total_endpoint_position'] = total_endpoint_position
    node_list['0']['connectivity'] = len(properties) 
    node_list['0']['total_mesh_number'] = mesh_numbering(skeleton_blurred_image)
    node_list['0']['total_thickness mean'] = thickness_mean
    node_list['0']['total_thickness std'] = thickness_std
    node_list['0']['total_vessel_image'] = binary_blurred_image
    node_list['0']['total_vessel_skel_image'] = skeleton_blurred_image

    
    for i in range(len(properties)) :
        single_connectivity_properties = { 'mesh_area': [], 'mesh_area_density': [], 'mesh_node_number': [], 'mesh_node_position': [], 'mesh_total_length': [], 'mesh_endpoint_number': [], 'mesh_endpoint_position': [], 'mesh_number': [], 'mesh_thickness (mean ± std)':[], 'total_single_node_length': [], 'mesh_image': [], 'mesh_skeleton_image': []  }
        node_list['{0}'.format(i+1)] = single_connectivity_properties
        mesh_node_position = find_branchpoints(properties[i]['image'])
        seperate_node = properties[i]['image']*1 - node_seperate(properties[i]['image'])*1
        divided_node = label(seperate_node, connectivity=2)
        single_node = regionprops(divided_node)
        thickness_mesh_mean, thickness_mesh_std = calculate_thickness(properties_vessel[i]['image'], morphology.skeletonize(properties_vessel[i]['image']))
        mesh_endpoint_number, mesh_endpoint_position = find_endpoints(properties[i]['image'])
        
        node_list['{0}'.format(i+1)]['mesh_area'] = properties_vessel[i]['area']
        node_list['{0}'.format(i+1)]['mesh_area_density'] = np.sum(properties_vessel[i]['image']) / properties_vessel[i]['image'].size
        node_list['{0}'.format(i+1)]['mesh_node_number'] = np.sum(mesh_node_position)
        node_list['{0}'.format(i+1)]['mesh_node_position'] = mesh_node_position
        node_list['{0}'.format(i+1)]['mesh_total_length'] = np.sum(properties[i]['image'])
        node_list['{0}'.format(i+1)]['mesh_endpoint_number'] = mesh_endpoint_number
        node_list['{0}'.format(i+1)]['mesh_endpoint_position'] = mesh_endpoint_position
        node_list['{0}'.format(i+1)]['mesh_number'] = mesh_numbering(properties[i]['image'])
        node_list['{0}'.format(i+1)]['mesh_thickness mean'] = thickness_mesh_mean
        node_list['{0}'.format(i+1)]['mesh_thickness std'] = thickness_mesh_std
        node_list['{0}'.format(i+1)]['mesh_image'] = properties_vessel[i]['image']
        node_list['{0}'.format(i+1)]['mesh_skeleton_image'] = properties[i]['image']

        for j in range(len(single_node)) :
            node_list['{0}'.format(i+1)]['total_single_node_length'].append(single_node[j]['area'])
   
    return node_list

In [6]:
### 25.05.16 vessel만 분석하고 click-erase 기능까지 담긴 알고리즘
import sys
import os
import cv2
import numpy as np
from skimage.filters import threshold_otsu, gaussian
from skimage import morphology
from skimage.morphology import dilation, disk, label
from skimage.measure import regionprops
import time
from PyQt5.QtWidgets import QApplication, QMainWindow, QLabel, QSlider, QVBoxLayout, QWidget, QPushButton, QHBoxLayout
from PyQt5.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import matplotlib.pyplot as plt
import pandas as pd
import copy
from PyQt5.QtWidgets import QLineEdit
from PyQt5.QtWidgets import QInputDialog, QMessageBox
from PyQt5.QtWidgets import QFileDialog, QMessageBox
import datetime
import os

class ImageProcessingApp(QMainWindow):
    def __init__(self):
        super().__init__()

        # ✅ 필수 변수들 초기화
        self.current_index = 0
        self.image_files = []
        self.saved_parameters = []  # 🔹 저장된 파라미터 리스트 초기화
        self.vasculo_analysis = {}  # 🔹 분석 결과 저장 딕셔너리 초기화

        # flood fill 관련 변수 (현재 이미지에 적용한 noise 제거 여부와 결과)
        self.flood_fill_applied = False
        self.binary_image_final = None

        # ✅ 사용자가 폴더를 선택할 수 있도록 GUI 실행 전에 select_folders() 실행
        if not self.select_folders():
            QMessageBox.warning(self, "Error", "No valid folder selected. Exiting...")
            return  # 🚀 GUI 종료 없이 함수만 종료

        # ✅ 첫 번째 이미지 로드 (self.image 설정)
        self.load_image()

        # ✅ UI 초기화
        self.initUI()

        # ✅ UI가 공백으로 나타나는 문제 해결
        QApplication.processEvents()


    def select_folders(self):
        """사용자가 분석할 이미지 폴더와 저장 폴더를 선택"""
        
        # ✅ 분석할 이미지 폴더 선택
        image_folder = QFileDialog.getExistingDirectory(self, "Select Image Folder")
        if not image_folder:
            return False  # 🚀 폴더 선택이 취소된 경우 False 반환

        # ✅ 모든 이미지 포맷을 검색 (jpg, png, tif, bmp 등)
        valid_extensions = ('.jpg', '.jpeg', '.png', '.tif', '.tiff', '.bmp')
        self.image_folder = image_folder
        self.image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith(valid_extensions)])

        if not self.image_files:
            QMessageBox.warning(self, "Warning", "No valid image files found. Please select another folder.")
            return False  # 🚀 이미지가 없는 경우 False 반환

        # ✅ 저장할 폴더 선택
        save_folder = QFileDialog.getExistingDirectory(self, "Select Save Folder")
        if not save_folder:
            return False  # 🚀 저장 폴더 선택이 취소된 경우 False 반환

        # ✅ UI가 공백 상태가 되지 않도록 강제 업데이트
        QApplication.processEvents()

        # 별도 객체화하여 팝업을 앞으로 띄우고 크기 조정
        input_dialog = QInputDialog(self)
        input_dialog.setInputMode(QInputDialog.TextInput)
        input_dialog.setWindowTitle("Enter File Name")
        input_dialog.setLabelText("Enter file name (default: HUVEC):")
        input_dialog.resize(400, 200)  # 크기를 키워줍니다 (원하는 크기로 변경 가능)
        input_dialog.setWindowModality(Qt.WindowModal)  # 메인 윈도우 위에 고정

        # 맨 앞으로 가져오기
        input_dialog.setWindowFlag(Qt.WindowStaysOnTopHint, True)

        # 화면 가운데 정렬 (선택적)
        frame_geo = input_dialog.frameGeometry()
        screen = QApplication.primaryScreen().availableGeometry().center()
        frame_geo.moveCenter(screen)
        input_dialog.move(frame_geo.topLeft())

        today = datetime.datetime.today().strftime("%y%m%d")  # 예: 250212
        self.base_filename = filename.strip()
        self.save_path = os.path.join(save_folder, f"{today} - {self.base_filename}")

        # ✅ 폴더가 없으면 생성
        os.makedirs(self.save_path, exist_ok=True)

        # ✅ UI 업데이트 강제 적용
        QApplication.processEvents()
        return True  # 🚀 폴더가 정상적으로 설정된 경우 True 반환

    
    def load_image(self):
        """현재 인덱스의 이미지 로드"""
        if self.current_index >= len(self.image_files):  # 마지막 이미지 이후면 분석 실행
            self.run_analysis()
            return

        image_path = os.path.join(self.image_folder, self.image_files[self.current_index])
        self.image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # 첫 이미지일 때만 기본 파라미터를 설정
        if self.current_index == 0:
            self.thresh_value = 10
            self.white_min = 100
            self.black_min = 200
            self.gaussian_blur = 3
            self.overlay_alpha = 0.4

        self.skeleton_image, self.binary_image = self.preprocessing_image()

        # flood fill 적용 관련 변수 초기화 (새 이미지 로드시 reset)
        self.flood_fill_applied = False
        self.binary_image_final = None


    def update_image(self):
        """이미지 업데이트"""
        # flood fill이 적용되었고, 수정된 binary 이미지가 있다면 그 결과를 이용해 skeleton 이미지도 업데이트
        if self.flood_fill_applied and self.binary_image_final is not None:
            binary_for_skeleton = self.binary_image_final  # boolean array (vessel: True, background: False)
            # flood fill 결과를 기반으로 skeleton 이미지 재계산
            skeleton_original = morphology.skeletonize(binary_for_skeleton)
            skeleton_dilated = dilation(skeleton_original, disk(2))
            self.skeleton_image = skeleton_dilated
            display_binary = self.binary_image_final
        else:
            # flood fill 적용 전: 원래의 전처리 결과 사용
            self.skeleton_image, self.binary_image = self.preprocessing_image()
            display_binary = self.binary_image

        # 왼쪽 axes: Original + Binary (색상은 원래대로 유지)
        self.axes[0].clear()
        self.axes[0].imshow(self.image, cmap='gray')
        self.axes[0].imshow(display_binary, cmap='jet', alpha=self.overlay_alpha)
        self.axes[0].set_title(f"Original + Binary ({self.image_files[self.current_index]})")

        # 오른쪽 axes: Original + Skeleton (flood fill 결과 반영)
        self.axes[1].clear()
        self.axes[1].imshow(self.image, cmap='gray')
        self.axes[1].imshow(self.skeleton_image, cmap='hot', alpha=self.overlay_alpha)
        self.axes[1].set_title(f"Original + Skeleton ({self.image_files[self.current_index]})")

        self.canvas.draw()

        

    def slider_changed(self, name, value, label):
        """마우스를 놓았을 때만 값 반영"""
        label.setText(f"{name}: {value}")
        # flood fill이 적용된 경우, 슬라이더 변경으로 binary 이미지를 재계산하면 flood fill 효과가 사라지므로 안내
        if self.flood_fill_applied:
            QMessageBox.information(self, "Notice", "Flood fill noise removal applied. To update parameters, please reset noise removal first.")
            return

        if name == "Threshold":
            self.thresh_value = value
        elif name == "White Min":
            self.white_min = value
        elif name == "Black Min":
            self.black_min = value
        elif name == "Gaussian Blur":
            self.gaussian_blur = value
        elif name == "Overlay Alpha":
            self.overlay_alpha = value / 100

        # ✅ 마우스를 놓은 후에만 update_image() 실행 (버벅임 방지)
        self.update_image()


    def initUI(self):
        """GUI 초기화: 파일명 입력 제거"""
        self.setWindowTitle("Image Processing GUI")
        self.setGeometry(100, 100, 2200, 1000)

        central_widget = QWidget()
        layout = QVBoxLayout()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # ✅ 파일명을 사용자에게 표시 (두 번 입력 방지)
        self.filename_label = QLabel(f"📁 Saving as: {self.base_filename}.xlsx")
        layout.addWidget(self.filename_label)

        # ✅ Matplotlib Figure 설정
        self.figure, self.axes = plt.subplots(1, 2, figsize=(18, 12), constrained_layout=True)
        self.canvas = FigureCanvas(self.figure)
        layout.addWidget(self.canvas)

        # ✅ imshow()를 처음 한 번만 호출 → 이후에는 update_image()로 재갱신
        dummy_image = np.zeros_like(self.image)
        self.img_binary = self.axes[0].imshow(dummy_image, cmap='jet', alpha=self.overlay_alpha)
        self.img_skeleton = self.axes[1].imshow(dummy_image, cmap='hot', alpha=self.overlay_alpha)

        self.axes[0].set_title("Original + Binary")
        self.axes[1].set_title("Original + Skeleton")

        self.canvas.draw()

        # ✅ 현재 이미지 번호 표시
        self.image_label = QLabel(f"Image {self.current_index + 1} of {len(self.image_files)}")
        layout.addWidget(self.image_label)

        # ✅ 슬라이더 및 조절 버튼 추가
        self.create_slider_with_buttons("Threshold", 0, 255, self.thresh_value, layout)
        self.create_slider_with_buttons("White Min", 0, 10000, self.white_min, layout)
        self.create_slider_with_buttons("Black Min", 0, 10000, self.black_min, layout)
        self.create_slider_with_buttons("Gaussian Blur", 0, 20, self.gaussian_blur, layout)
        self.create_slider_with_buttons("Overlay Alpha", 0, 100, int(self.overlay_alpha * 100), layout)

        # ✅ "Next Image" 버튼 추가
        self.next_button = QPushButton("Next Image")
        self.next_button.clicked.connect(self.next_image)  # 클릭 시 다음 이미지 로드
        layout.addWidget(self.next_button)
        
        # ✅ "Reset Noise Removal" 버튼 추가 (flood fill 적용 취소)
        self.reset_noise_button = QPushButton("Reset Noise Removal")
        self.reset_noise_button.clicked.connect(self.reset_flood_fill)
        layout.addWidget(self.reset_noise_button)

        # ✅ Matplotlib canvas에 마우스 클릭 이벤트 연결 (왼쪽 이미지에서 노이즈 제거)
        self.canvas.mpl_connect('button_press_event', self.on_click)

        self.update_image()  # 초기 이미지 표시


    def create_slider_with_buttons(self, name, min_val, max_val, init_val, layout):
        """슬라이더와 함께 + / - 버튼 추가"""
        hbox = QHBoxLayout()

        label = QLabel(f"{name}: {init_val}")
        hbox.addWidget(label)

        minus_button = QPushButton("-")
        minus_button.setFixedWidth(40)
        minus_button.clicked.connect(lambda: self.adjust_slider(name, -1, label))
        hbox.addWidget(minus_button)

        slider = QSlider(Qt.Horizontal)
        slider.setMinimum(min_val)
        slider.setMaximum(max_val)
        slider.setValue(init_val)

        # ✅ 슬라이더를 움직일 때 즉시 반영하지 않음
        slider.valueChanged.connect(lambda val: label.setText(f"{name}: {val}"))

        # ✅ 슬라이더를 놓았을 때만 업데이트 실행
        slider.sliderReleased.connect(lambda: self.slider_changed(name, slider.value(), label))

        hbox.addWidget(slider)

        plus_button = QPushButton("+")
        plus_button.setFixedWidth(40)
        plus_button.clicked.connect(lambda: self.adjust_slider(name, 1, label))
        hbox.addWidget(plus_button)

        layout.addLayout(hbox)

        # ✅ 슬라이더 변수를 명확히 인스턴스 변수로 저장 (AttributeError 해결)
        if name == "Threshold":
            self.thresh_slider = slider
        elif name == "White Min":
            self.white_min_slider = slider
        elif name == "Black Min":
            self.black_min_slider = slider
        elif name == "Gaussian Blur":
            self.gaussian_blur_slider = slider
        elif name == "Overlay Alpha":
            self.overlay_alpha_slider = slider


    def adjust_slider(self, name, step, label):
        """+ / - 버튼 클릭 시 슬라이더 값을 조정"""
        # "Threshold" 슬라이더의 경우 인스턴스 이름을 특별 처리
        if name == "Threshold":
            slider = self.thresh_slider
        else:
            slider = getattr(self, f"{name.lower().replace(' ', '_')}_slider")
        new_value = max(slider.minimum(), min(slider.maximum(), slider.value() + step))
        slider.setValue(new_value)  # 슬라이더 값 업데이트
        label.setText(f"{name}: {new_value}")
        
        # ± 버튼 클릭 시 바로 이미지 업데이트를 실행하도록 변경
        self.slider_changed(name, new_value, label)


    def preprocessing_image(self):
        """이미지 전처리 수행"""
        binary_image = self.image > self.thresh_value
        cleaned_image = morphology.remove_small_objects(binary_image, min_size=self.white_min)
        inverted_image = np.logical_not(cleaned_image)
        filtered_black_holes = morphology.remove_small_objects(inverted_image, min_size=self.black_min)
        final_binary_image = np.logical_not(filtered_black_holes)

        blurred_image = gaussian(final_binary_image, sigma=self.gaussian_blur)
        thresh_value_blurred = threshold_otsu(blurred_image)
        binary_blurred_image = blurred_image > thresh_value_blurred
        
        skeleton_original = morphology.skeletonize(binary_blurred_image)
        skeleton_dilated = dilation(skeleton_original, disk(2))

        self.skeleton_original = skeleton_original

        return skeleton_dilated, binary_blurred_image


    def on_click(self, event):
        """마우스 클릭 시 flood fill을 통해 노이즈 제거 (왼쪽 axes에 한정)"""
        if event.inaxes == self.axes[0]:
            x, y = int(event.xdata), int(event.ydata)
            print(f"Flood fill click at: ({x}, {y})")
            
            # flood fill 적용 전, 이전에 flood fill 결과가 있다면 그 결과를 기반으로 작업,
            # 없으면 원래 전처리된 binary 이미지를 사용
            if self.binary_image_final is None:
                working_img = (self.binary_image.astype(np.uint8)) * 255
            else:
                working_img = (self.binary_image_final.astype(np.uint8)) * 255
            
            h, w = working_img.shape
            mask = np.zeros((h + 2, w + 2), np.uint8)
            
            # floodFill 파라미터 (필요에 따라 조정)
            new_color = 0
            loDiff = 10
            upDiff = 10
            
            # flood fill 실행 (working_img 복사본에서)
            retval, filled_img, mask_ret, rect = cv2.floodFill(working_img.copy(), mask, (x, y), new_color,
                                                                    (loDiff,)*3, (upDiff,)*3)
            print(f"Flood fill applied, {retval} 픽셀 변경됨.")
            
            # filled_img의 값이 255인 픽셀은 vessel(원래 값)을 의미하므로, 이를 그대로 boolean으로 변환
            filled_binary = (filled_img == 255)
            
            # flood fill 결과를 self.binary_image_final에 저장 (누적 적용됨)
            self.binary_image_final = filled_binary
            self.flood_fill_applied = True
            
            self.update_image()



    def reset_flood_fill(self):
        """Flood fill 적용 취소: 원래 전처리된 binary 이미지로 복원"""
        self.flood_fill_applied = False
        self.binary_image_final = None
        self.update_image()


    def next_image(self):
        """다음 이미지로 전환"""
        # ✅ 현재 이미지에 대한 파라미터 저장을 이미지 로드 전에 수행하도록 수정
        self.saved_parameters.append({
            "thresh_value": self.thresh_slider.value(),
            "white_min": self.white_min_slider.value(),
            "black_min": self.black_min_slider.value(),
            "gaussian_blur": self.gaussian_blur_slider.value(),
            "overlay_alpha": self.overlay_alpha_slider.value() / 100
        })

        # ✅ 현재 오버랩된 이미지를 저장
        self.save_overlay_image(self.current_index + 1)

        # ✅ 마지막 이미지 초과 시 분석 실행
        if self.current_index + 1 >= len(self.image_files):
            self.run_analysis()
            return

        # ✅ 다음 이미지로 이동 (flood fill 변수 초기화)
        self.current_index += 1  # 🔹 이미지를 바꾸기 전에 index 증가
        self.load_image()  # 🔹 새로운 이미지를 로드

        # ✅ 기존 슬라이더 값을 유지하도록 변경 (새로운 이미지에 대한 슬라이더 값 적용)
        if len(self.saved_parameters) > self.current_index:
            last_params = self.saved_parameters[self.current_index]
            self.thresh_slider.setValue(last_params["thresh_value"])
            self.white_min_slider.setValue(last_params["white_min"])
            self.black_min_slider.setValue(last_params["black_min"])
            self.gaussian_blur_slider.setValue(last_params["gaussian_blur"])
            self.overlay_alpha_slider.setValue(int(last_params["overlay_alpha"] * 100))

        self.update_image()
        self.image_label.setText(f"Image {self.current_index + 1} of {len(self.image_files)}")


    def save_overlay_image(self, base_index):
        """현재 GUI에 표시된 오버랩 이미지를 PNG로 저장"""
        # ✅ 파일 경로 설정 (Threshold & Skeleton)
        threshold_filename = os.path.join(self.save_path, f"{self.base_filename} - {base_index} - Threshold overlap.png")
        skeleton_filename = os.path.join(self.save_path, f"{self.base_filename} - {base_index} - Skeleton overlap.png")

        # ✅ Threshold overlap 저장 (왼쪽 이미지)
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.imshow(self.image, cmap='gray')
        # flood fill 적용된 경우 저장할 때는 수정된 binary_image_final 사용
        if self.flood_fill_applied and self.binary_image_final is not None:
            ax.imshow(self.binary_image_final, cmap='jet', alpha=self.overlay_alpha)
        else:
            ax.imshow(self.binary_image, cmap='jet', alpha=self.overlay_alpha)
        ax.set_title("Threshold Overlap")
        ax.axis("off")
        plt.savefig(threshold_filename, dpi=300, bbox_inches='tight')
        plt.close(fig)

        # ✅ Skeleton overlap 저장 (오른쪽 이미지)
        fig, ax = plt.subplots(figsize=(5, 5))
        ax.imshow(self.image, cmap='gray')
        ax.imshow(self.skeleton_image, cmap='hot', alpha=self.overlay_alpha)
        ax.set_title("Skeleton Overlap")
        ax.axis("off")
        plt.savefig(skeleton_filename, dpi=300, bbox_inches='tight')
        plt.close(fig)

        print(f"✅ Saved overlay images:\n  {threshold_filename}\n  {skeleton_filename}")


    def run_analysis(self):
        """모든 이미지 확인 후 저장된 파라미터를 사용하여 분석 실행"""
        self.next_button.setText("Analyzing...")
        self.next_button.setEnabled(False)
        QApplication.processEvents()

        total_images = len(self.saved_parameters)
        analysis_results = []
        image_mesh_data = {}

        for i, params in enumerate(self.saved_parameters):
            self.image_label.setText(f"Analyzing {i+1}/{total_images}")
            QApplication.processEvents()

            # ✅ 개별 이미지 로드
            image_path = os.path.join(self.image_folder, self.image_files[i])
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 🚀 개별 이미지 로드

            # ✅ 저장된 개별 이미지 파라미터를 정확히 반영
            thresh_value = params["thresh_value"]
            white_min = params["white_min"]
            black_min = params["black_min"]
            gaussian_blur = params["gaussian_blur"]

            # ✅ 전처리 적용 후 정량 분석 (🚀 self.preprocessing_image()가 아니라 독립적인 함수 사용)
            skeleton_dilated, binary, skeleton_original = preprocessing_image(
                image, thresh_value, white_min, black_min, gaussian_blur
            )
            analysis = node_analysis(skeleton_original, binary)
            self.vasculo_analysis[i] = analysis  # 딕셔너리에 저장

            # ✅ Total Data 저장 (기존 문제 수정: 개별 이미지의 파라미터와 결과를 정확히 저장)
            analysis_results.append({
                "Image": self.image_files[i],
                "Threshold": thresh_value,
                "White Min": white_min,
                "Black Min": black_min,
                "Gaussian Blur": gaussian_blur,
                "Total Area": analysis["0"]["total_area"],
                "Total Density": analysis["0"]["total_density"],
                "Total Node Number": analysis["0"]["total_node_number"],
                "Total Node Length": analysis["0"]["total_node_length"],
                "Total Endpoint Number": analysis["0"]["total_endpoint_number"],
                "Connectivity": analysis["0"]["connectivity"],
                "Total Mesh Number": analysis["0"]["total_mesh_number"],
                "Total Thickness Mean": analysis["0"]["total_thickness mean"],
                "Total Thickness Std": analysis["0"]["total_thickness std"]
            })

            # ✅ 개별 mesh 데이터를 저장
            mesh_list = []
            for key, mesh in analysis.items():
                if key == "0":  # 첫 번째 항목은 Total Data이므로 스킵
                    continue
                mesh_list.append({
                    "Mesh Area": mesh["mesh_area"],
                    "Mesh Area Density": mesh["mesh_area_density"],
                    "Mesh Node Number": mesh["mesh_node_number"],
                    "Mesh Total Length": mesh["mesh_total_length"],
                    "Mesh Endpoint Number": mesh["mesh_endpoint_number"],
                    "Mesh Number": mesh["mesh_number"],
                    "Mesh Thickness Mean": mesh["mesh_thickness mean"],
                    "Mesh Thickness Std": mesh["mesh_thickness std"]
                })

            # ✅ Mesh 데이터를 크기순으로 정렬하여 저장
            image_mesh_data[f"{self.base_filename}-{i+1}"] = sorted(mesh_list, key=lambda x: x["Mesh Area"], reverse=True)

        # ✅ 엑셀 저장 (다중 시트 지원)
        excel_path = os.path.join(self.save_path, f"{self.base_filename}.xlsx")
        with pd.ExcelWriter(excel_path, engine="xlsxwriter") as writer:
            df_total = pd.DataFrame(analysis_results)
            df_total.to_excel(writer, sheet_name="Total Data", index=False)

            for sheet_name, mesh_data in image_mesh_data.items():
                df_mesh = pd.DataFrame(mesh_data)
                df_mesh.to_excel(writer, sheet_name=sheet_name, index=False)

        print(f"✅ 분석 결과가 엑셀 파일로 저장되었습니다: {excel_path}")
         # 🚀 **분석 완료 후 팝업 띄우기**
        QMessageBox.information(self, "Analysis Complete", "✅ 분석이 완료되었습니다!\n저장된 폴더를 확인하세요.")


    def closeEvent(self, event):
        """창을 닫을 때 안전하게 종료"""
        print("Closing application...")
        self.figure.clear()
        plt.close(self.figure)
        QApplication.quit()
        event.accept()


app = QApplication.instance()  # ✅ 기존 QApplication 인스턴스가 있으면 재사용
if app is None:
    app = QApplication(sys.argv)  # 새로운 QApplication 생성

window = ImageProcessingApp()
window.show()

# ✅ Jupyter Notebook에서 실행할 때 sys.exit() 제거
app.exec_()  # GUI 실행 후, 커널 유지 (변수 초기화 방지)

# ✅ 분석 결과를 유지하여 확인 가능
vasculo_analysis = window.vasculo_analysis
saved_parameters = window.saved_parameters

print("✅ GUI 종료 후에도 변수 유지됨!")

Closing application...


AttributeError: 'ImageProcessingApp' object has no attribute 'figure'

✅ GUI 종료 후에도 변수 유지됨!
